In [1]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr()

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # 5 наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df['High Correlation Count'] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_pearson_correlation_with_label.csv",
        index_label="Packet Interval",
    )

process_attack_data("sql-inj", "./csvs/")

Correlation table for attack type: sql-inj
     Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                 0.061         0.097          0.145              0.131        0.200        0.194                  0.222                   0.222              0.062                        0.062                       0
50                 0.073         0.005          0.148              0.306        0.235        0.356                  0.249                   0.249              0.070                        0.070                       0
100                0.125         0.094          0.251              0.349        0.089        0.534                  0.439                   0.439              0.121                        0.121                       0
250                0.162         0.664          0.775              0.985          NaN

In [2]:
import os
import pandas as pd


# Функция для обработки данных одной атаки
def process_attack_data(attack_type, csvs_folder):
    # Списки для нормального и атакующего трафика
    normal_files = os.listdir(os.path.join(csvs_folder, "normal"))
    attack_files = os.listdir(os.path.join(csvs_folder, "attack"))

    # Словарь для хранения корреляций по пакетным интервалам
    correlation_data = {}

    # Для каждого интервала пакетных выборок
    for normal_file in normal_files:
        # Определяем соответствующий файл для атаки
        packet_size = normal_file.split("_")[1]
        attack_file = [f for f in attack_files if f"_{packet_size}_" in f]

        if attack_file:
            # Чтение нормального и атакующего трафика
            normal_df = pd.read_csv(os.path.join(csvs_folder, "normal", normal_file))
            attack_df = pd.read_csv(os.path.join(csvs_folder, "attack", attack_file[0]))

            # Объединяем нормальные и атакующие данные
            combined_df = pd.concat([normal_df, attack_df])

            # Вычисление корреляционной матрицы
            corr_matrix = combined_df.corr(method="spearman")

            # Выделение корреляции с меткой "Label"
            target_corr = corr_matrix["Label"].abs().sort_values(ascending=False)

            # Сохраняем топ-5 коррелирующих признаков (исключая 'Label')
            top_features = target_corr.index[1:]  # N наиболее значимых признаков

            # Сохраняем корреляции для текущего интервала
            correlation_data[packet_size] = target_corr[top_features]

    # Преобразуем словарь в DataFrame, строки - это интервал пакетов, столбцы - признаки
    correlation_df = pd.DataFrame(
        correlation_data
    ).T  # Транспонируем для правильной формы

    # Преобразуем индекс из строк в числа для корректной сортировки
    correlation_df.index = correlation_df.index.astype(int)

    # Добавляем столбец с количеством признаков, имеющих корреляцию больше 0.7
    correlation_df["High Correlation Count"] = (correlation_df > 0.7).sum(axis=1)

    # Округление значений до 3 знаков после запятой
    correlation_df = correlation_df.round(3)

    # Сортировка по индексу (длине пакетной выборки)
    correlation_df = correlation_df.sort_index()

    # Вывод таблицы на экран
    print(f"Correlation table for attack type: {attack_type}")
    print(correlation_df.to_string(index=True))

    # Сохраняем DataFrame в CSV файл
    correlation_df.to_csv(
        f"{attack_type}_spearman_correlation_with_label.csv",
        index_label="Packet Interval",
    )


process_attack_data("sql-inj", "./csvs/")

Correlation table for attack type: sql-inj
     Average Packet Size  Flow Bytes/s  Flow IAT Mean  Fwd Header Length  Fwd IAT Min  Fwd IAT Std  Fwd Packet Length Max  Fwd Packet Length Mean  Max Packet Length  Total Length of Fwd Packets  High Correlation Count
10                 0.068         0.090          0.076              0.221        0.172        0.096                  0.109                   0.109              0.071                        0.071                       0
50                 0.026         0.191          0.035              0.395        0.053        0.122                  0.247                   0.247              0.035                        0.035                       0
100                0.342         0.342          0.266              0.305        0.089        0.342                  0.393                   0.393              0.342                        0.342                       0
250                0.258         0.775          0.775              0.775          NaN